In [1]:
from time import gmtime, strftime
import numpy as np
import pandas as pd
import os
import dicom
from report_parser import parse_report
import pickle 
import tqdm
import random

/scratch/cxr-baselines/cxr-baselines/venv_cxr/lib/python3.5/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
# Get train and test data
data_dir = '/scratch/wboag/2019/cxr/cxr-baselines/camera_ready/data'
train_df = pd.read_csv(os.path.join(data_dir,'train.tsv'), sep='\t')
test_df  = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')

print(train_df.shape)
display(train_df.head())

print(test_df.shape)
display(test_df.head())

(228136, 7)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,subject_id,rad_id,dicom_id,dicom_is_available
0,0,0,2,70233355,53378012,3108d905-782ffdc0-209309e8-2413eeb4-6bfb958a,True
1,1,1,3,70233355,53378012,40eab5a8-31446771-08c6b024-2717a65c-41f8c74f,True
2,2,2,6,78564939,51423061,fc601540-ae89d087-3589ac06-85224a6b-bb5960ce,True
3,3,3,7,78564939,51423061,9956b6ce-67a4e84b-6038ce80-52428d83-04d83f25,True
4,4,4,8,71322,51527637,d5072bc4-bb422de8-97f3973a-0d8e5ae0-7c52ac3b,True


(99145, 7)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,subject_id,rad_id,dicom_id,dicom_is_available
0,0,0,27,73451872,55507110,f1a7e903-618a45fe-84eb71e2-73901894-a689d584,True
1,1,1,28,73451872,55507110,523dbd29-f0c5d7eb-09635cf1-1a7de126-44622b1c,True
2,2,2,37,68870,51526655,78ecaf71-9fdb0b43-b0134402-8c5e739f-2c6c0ea2,True
3,3,3,38,68870,51526655,14089000-1023e4ed-157da1b0-f14f1dcd-7eaf3cb2,True
4,4,4,39,68870,57395479,ceb36d05-686e9404-43dfdc4f-e050bf09-89b8d71d,True


In [3]:
# Map each dicom to its rad_id
rad_lookup = dict(train_df[['dicom_id','rad_id']].values)
dict(list(rad_lookup.items())[:5])

{'1ebdbf50-86954fde-8df90ca4-4ba64744-6e968542': 51396613,
 '70435869-e759a079-121fd1c4-c49a7d27-4e8e9dda': 55088759,
 'aba507af-093be34b-239c19e6-72ba3cb4-5bbe84c7': 57170456,
 'bc3616ab-15422ebc-86477c1b-a15c35ee-baf33823': 54084658,
 'd95d1c39-06a8bf97-1e6d2ff2-a9918b85-c0e80a4a': 54150236}

In [4]:
# Where reports are stored
reports_path = '/crimea/mimic-cxr/reports'

In [5]:
generated_reports = {}
for pred_dicom in tqdm.tqdm(test_df.dicom_id):
    #print(pred_dicom)
    found = False
    while not found:
        nearest_dicom = random.choice(train_df.dicom_id)
        nearest_train_rad_id = rad_lookup[nearest_dicom]
        #print('\t', nearest_dicom)
        
        # Get text of report
        nearest_train_report_file = os.path.join(reports_path, '%s.txt' % nearest_train_rad_id)
        report = parse_report(nearest_train_report_file)
        
        # If the report doesn't have a findings section, then get the next-closest report
        if 'findings' in report:
            found = True
        
    #break
    generated_reports[pred_dicom] = report['findings']


100%|██████████| 99145/99145 [00:25<00:00, 3881.69it/s]


In [6]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

pred_dir = 'output'
pred_file = os.path.join(pred_dir, 'random.tsv')
print(pred_file)
with open(pred_file, 'w') as f:
    print('dicom_id\tgenerated', file=f)
    for dicom_id,generated in sorted(generated_reports.items()):
        print('%s\t%s' % (dicom_id,generated), file=f)
        
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2019-11-13 02:29:24
output/random.tsv
2019-11-13 02:29:24
